In [1]:
import silencer
from initializer import initData
from gradient import nlseGradient
from network import physicsNetwork
import numpy as np
import tensorflow as tf


# Debugging

## initData class

In [2]:
'''
debugging initData class
'''
#number of samples
num_train = 1000
num_test = 1000

#time and space domain
t_f = 5
x_f = 2 
x_init = -2

#traning input
##1. random collocation point
col_point = np.random.rand(num_train, 2)
col_point[..., 0] = t_f*col_point[..., 0] #temporal t domain
col_point[..., 1] = (x_f-x_init)*col_point[..., 1] + x_init #spatial x domain

##2. Initial f(t,x) array

def u0(tx):
    t = tx[..., 0, None]
    x = tx[..., 1, None]
    return np.exp(-x**2)
init_point = np.random.rand(num_train, 2)
init_point[..., 0] = t_f*init_point[..., 0] #temporal t domain
init_point[..., 1] = (x_f-x_init)*init_point[..., 1] + x_init #spatial x domain

u_init = u0(init_point)
v_init = np.zeros((num_train, 1))

##3. Boundary f(t,x) array
bound_point_up = np.random.rand(num_train, 2)
bound_point_up[..., 0] = t_f*bound_point_up[..., 0] #temporal t domain
bound_point_up[..., 1] = x_f #spatial x domain

bound_point_down = np.random.rand(num_train, 2)
bound_point_down[..., 0] = t_f*bound_point_down[..., 0] #temporal t domain
bound_point_up[..., 1] = x_init #spatial x domain

u_bound = np.zeros((num_train, 1))
v_bound = np.zeros((num_train, 1))

#Initialize and Batch Data
preparation = initData()
preparation.add_collocation_point(col_point)
preparation.add_labelled_data(init_point, u_init, v_init, 'init_point')
preparation.add_labelled_data(bound_point_up, u_bound, v_bound, 'bound_up')
preparation.add_labelled_data(bound_point_down, u_bound, v_bound, 'bound_down')

batched_data = preparation.batch_data()
keys = preparation.get_keys()

for key in keys:
    print(f'{key}:{len(batched_data[key])}')


col_point:10
init_point:10
bound_up:10
bound_down:10


## physicsNetwork class

In [3]:
dataset = batched_data['col_point']

for batch in dataset:
    array1, array2, array3 = batch
    break


In [4]:
model = physicsNetwork.build()
model(array1)[...,0]

<tf.Tensor: shape=(100,), dtype=float32, numpy=
array([-0.30407476,  0.24667335,  0.21904299, -0.29122338,  0.08311113,
        0.2256339 ,  0.04010938, -0.00851935, -0.22792432,  0.26407644,
        0.1340629 ,  0.05028082, -0.13813128, -0.28314015,  0.01792972,
        0.24062829, -0.3247753 ,  0.03228272, -0.22120655,  0.22004618,
        0.11860322, -0.23544115,  0.21542837,  0.19841295,  0.28044724,
       -0.19349696,  0.11124263, -0.23829292, -0.22869995,  0.16895556,
        0.27963617, -0.22212353,  0.06428463, -0.14687975, -0.19082643,
       -0.01139907, -0.19963747, -0.11175173, -0.13897723, -0.0070363 ,
        0.07388267, -0.1112051 , -0.00877967,  0.05780257,  0.14476222,
       -0.05806398,  0.21701132, -0.11471391, -0.06025393, -0.01925865,
       -0.07665911, -0.02631295,  0.04151922, -0.11934691, -0.20742996,
       -0.00994336, -0.28564376, -0.23755556, -0.26688597, -0.3167264 ,
        0.20952559,  0.167249  , -0.05423862, -0.29701126, -0.21804415,
        0.239018

## nlseGradient class

In [ ]:
nlseGrads = nlseGradient(model)
deriv_u, deriv_v = nlseGrads.call(array1)
deriv_u[0]